In [1]:
# !pip install tensorflow

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import pandas as pd
from keras.models import load_model 

2023-04-15 08:30:30.914437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-15 08:30:31.146660: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-15 08:30:31.150100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 08:30:32.000848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
lista_nok_borda = os.listdir("./assets/NOK_borda/")
# lista_nok_superficie = os.listdir("./assets/NOK_superficie/")
lista_nok_tamanho_forma = os.listdir("./assets/NOK_tamanho_forma/")
lista_ok = os.listdir("./assets/OK/")

lista_nok_borda = [f"./assets/NOK_borda/{filename}" for filename in lista_nok_borda]
# lista_nok_superficie = [f"./assets/NOK_superficie/{filename}" for filename in lista_nok_superficie]
lista_nok_tamanho_forma = [f"./assets/NOK_tamanho_forma/{filename}" for filename in lista_nok_tamanho_forma]
lista_ok = [f"./assets/OK/{filename}" for filename in lista_ok]

lista_total = []
lista_total.extend(lista_nok_borda)
# lista_total.extend(lista_nok_superficie)
lista_total.extend(lista_nok_tamanho_forma)
lista_total.extend(lista_ok)

lista_total

['./assets/NOK_borda/Fig_NOK_09.jpg',
 './assets/NOK_borda/Fig_NOK_11.jpg',
 './assets/NOK_borda/Fig_NOK_13.jpg',
 './assets/NOK_borda/Fig_NOK_14.jpg',
 './assets/NOK_borda/Fig_NOK_12.jpg',
 './assets/NOK_borda/Fig_NOK_08.jpg',
 './assets/NOK_borda/Fig_NOK_10.jpg',
 './assets/NOK_borda/Fig_NOK_06.jpg',
 './assets/NOK_borda/Fig_NOK_15.jpg',
 './assets/NOK_borda/Fig_NOK_07.jpg',
 './assets/NOK_tamanho_forma/Fig_NOK_05.jpg',
 './assets/NOK_tamanho_forma/Fig_NOK_04.jpg',
 './assets/NOK_tamanho_forma/Fig_NOK_02.jpg',
 './assets/NOK_tamanho_forma/Fig_NOK_03.jpg',
 './assets/NOK_tamanho_forma/Fig_NOK_01.jpg',
 './assets/OK/Fig_OK_09.jpg',
 './assets/OK/Fig_OK_08.jpg',
 './assets/OK/Fig_OK_04.jpg',
 './assets/OK/Fig_OK_10.jpg',
 './assets/OK/Fig_OK_15.jpg',
 './assets/OK/Fig_OK_06.jpg',
 './assets/OK/Fig_OK_05.jpg',
 './assets/OK/Fig_OK_07.jpg',
 './assets/OK/Fig_OK_11.jpg',
 './assets/OK/Fig_OK_01.jpg',
 './assets/OK/Fig_OK_02.jpg',
 './assets/OK/Fig_OK_14.jpg',
 './assets/OK/Fig_OK_03.jpg',


## Borda

In [ ]:
def fillHoles(img_thr):

    """
    Prenche furos internos em blobs, que deve ser branco (intensidade=255)

    Keyword arguments:
    img_thr -- nparray uint8 da imagem de entrada (imagem binarizada)

    Returns:
    img_out -- npaarray (uint8) da imagem apos preenchimento dos furos

    """

    img_floodfill = img_thr.copy()
    h, w = img_thr.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(img_floodfill, mask, (0,0), 255)
    img_floodfill_inv = cv2.bitwise_not(img_floodfill)
    img_out = img_thr | img_floodfill_inv

    return img_out

In [ ]:
def calcula_diametro(comprimento_px, largura_real_cm, comprimento_obj):

    # mede o comprimento da imagem em pixels
    comprimento_px = comprimento_px

    # calcula a relação de pixels por centímetro (ppc)
    ppc = comprimento_px / largura_real_cm

    # mede o comprimento de um objeto na imagem em pixels
    comprimento_objeto_px = comprimento_obj

    # converte o comprimento em pixels para centímetros
    comprimento_objeto_cm = comprimento_objeto_px / ppc

    raio = comprimento_objeto_cm/10
    diametro = 2*comprimento_objeto_cm/10

    # exibe o comprimento do objeto em centímetros
#     print('Raio do objeto:', raio, 'mm')
#     print('Diametro do objeto:', diametro, 'mm')

    nominal = 50
    tolerancia = 1

    return raio


In [ ]:
50*(0.99)

In [ ]:
50*(1.01)

In [ ]:
def calcula_relacao_eixo_a_e_b(ellipse):
    a, b = ellipse[1][0]/2, ellipse[1][1]/2
    teste_a_b = a/b
    
    return teste_a_b

In [ ]:
df = pd.DataFrame(
    columns=["FILENAME", "TESTE - BORDA", "TESTE - SUPERFICIE", "DIAMETRO", "STATUS - DIAMETRO", "A/B", "STATUS - A/B"]
)
for path in lista_total:
    TESTE_BORDA = "-"
    TESTE_SUPERFICIE = "-"
    DIAMETRO = "-"
    AB = "-"
    STATUS_DIAMETRO = "-"
    STATUS_AB = "-"
    FILENAME = path
    
    fig_borda = cv2.imread(path)

    # Separa os canais HSV
    hsv = cv2.cvtColor(fig_borda, cv2.COLOR_BGR2HSV)

    h,s,v = cv2.split(hsv)

    hsv_min = np.array([120, 0, 0])
    hsv_max = np.array([255, 150, 95])

    mask = cv2.inRange(fig_borda, hsv_min, hsv_max)

    result = cv2.bitwise_and(fig_borda, fig_borda, mask=~mask)

    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    _, fig_gray = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    
    blur = cv2.medianBlur(fig_gray, 5)
    
#     kp = detector.detect(blur)

    contours, _ = cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    cnt = max(contours, key = cv2.contourArea)
    ellipse = cv2.fitEllipse(cnt)
    
    _, radius = cv2.minEnclosingCircle(cnt)

    area_contour = cv2.contourArea(cnt)
    area_ellipse = np.pi * ellipse[1][0] * ellipse[1][1] / 4.0
    diff = area_ellipse - area_contour
        
    if(diff > 20):
        TESTE_BORDA = "Reprovado"
        data = {
           "FILENAME": [FILENAME],
           "TESTE - BORDA": [TESTE_BORDA],
           "TESTE - SUPERFICIE": [TESTE_SUPERFICIE],
           "DIAMETRO": [DIAMETRO],
           "STATUS - DIAMETRO": [STATUS_DIAMETRO],
           "A/B": [AB],
           "STATUS - A/B": [STATUS_AB]
        }
        data = pd.DataFrame(data)
        df = pd.concat([df, data], ignore_index=True)
        continue
    else:
        TESTE_BORDA = "Aprovado"
        
    
    contagem_pixels = []

    # Percorre cada linha da imagem
    for y in range(blur.shape[0]):
        # Conta o número de pixels 255 na linha atual
        contagem = cv2.countNonZero(blur[y])
        # Adiciona o número de pixels contados à lista
        contagem_pixels.append(contagem)

    # Encontra a linha com o maior número de pixels 255
    linha_mais_pixels = max(contagem_pixels)
    diametro = calcula_diametro(508, 650, linha_mais_pixels)
    if(diametro > 49.5 and diametro < 50.5):
        STATUS_DIAMETRO = "Aprovado"
        DIAMETRO = diametro
    else:
        STATUS_DIAMETRO = "Reprovado"
        DIAMETRO = diametro

    ab = calcula_relacao_eixo_a_e_b(ellipse)
    if(ab > 0.95 and ab < 1.05):
        AB = ab
        STATUS_AB = "Aprovado"
    else:
        AB = ab
        STATUS_AB = "Reprovado"
        
    data = {
       "FILENAME": [FILENAME],
       "TESTE - BORDA": [TESTE_BORDA],
       "TESTE - SUPERFICIE": [TESTE_SUPERFICIE],
       "DIAMETRO": [DIAMETRO],
       "STATUS - DIAMETRO": [STATUS_DIAMETRO],
       "A/B": [AB],
       "STATUS - A/B": [STATUS_AB],
    }
    data = pd.DataFrame(data)
    df = pd.concat([df, data], ignore_index=True)
df

In [ ]:
df["TESTE - BORDA"].value_counts()

In [ ]:
fig_borda = cv2.imread("./assets/NOK_borda/Fig_NOK_06.jpg")

# Separa os canais HSV
hsv = cv2.cvtColor(fig_borda, cv2.COLOR_BGR2HSV)

h,s,v = cv2.split(hsv)

# hsv_min = np.array([120, 0, 0])
# hsv_max = np.array([255, 150, 95])

hsv_min = np.array([110, 0, 0])
hsv_max = np.array([255, 150, 72])

mask = cv2.inRange(fig_borda, hsv_min, hsv_max)

result = cv2.bitwise_and(fig_borda, fig_borda, mask=~mask)

gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

_, fig_gray = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

blur = cv2.GaussianBlur(fig_gray, (5,5), 0.7, 0.7)

contours, _ = cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

cnt = max(contours, key = cv2.contourArea)
ellipse = cv2.fitEllipse(cnt)

area_contour = cv2.contourArea(cnt)
area_ellipse = np.pi * ellipse[1][0] * ellipse[1][1] / 4.0
diff = area_ellipse - area_contour
# print(area_contour, area_ellipse,diff, path)
(x,y), radius = cv2.minEnclosingCircle(cnt)
center = (int(x),int(y))
area_contour = cv2.contourArea(cnt)
area_ellipse = np.pi * ellipse[1][0] * ellipse[1][1] / 4.0
diff = area_ellipse - area_contour

raio = calcula_diametro(comprimento_px=fig_borda.shape[1], largura_real_cm=650, comprimento_obj=radius)
print(raio)

cv2.drawContours(fig_borda,[cnt], -1, (0,0,255), 2)
# cv2.circle(fig_borda, center, int(radius), (0, 255, 0), 2)
cv2.ellipse(fig_borda, ellipse, (0, 255, 0), 2)

plt.figure(figsize=(21,21))
plt.subplot(1,3,1)
plt.imshow(fig_borda[:,:,::-1])
plt.subplot(1,3,2)
plt.imshow(result[:,:,::-1])

In [ ]:
for path in lista_nok_borda:
    fig_borda = cv2.imread(path)
    
    # dividir a imagem em seus canais de cores
    b, g, r = cv2.split(fig_borda)

    # criar uma matriz nula para os outros canais
    zeros = np.zeros_like(b)

    # mesclar os canais para obter somente o canal azul
    blue_img = cv2.merge((zeros, g, zeros))

    x = cv2.cvtColor(blue_img, cv2.COLOR_BGR2GRAY)

    _, fig_th = cv2.threshold(x, 41, 255, cv2.THRESH_BINARY_INV)
    
    fig_th = cv2.medianBlur(fig_th, 5)

    contours, _ = cv2.findContours(fig_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    cnt = max(contours, key = cv2.contourArea)
    area = cv2.contourArea(cnt)

    # Calcula o diâmetro do objeto
    _, radius = cv2.minEnclosingCircle(cnt)
    diameter = 2 * radius

    # Calcula a área de um círculo com o mesmo diâmetro do objeto
    circle_area = np.pi * (diameter/2)**2

    # Calcula a relação entre a área do objeto e a área do círculo
    circularity = area / circle_area

    print(circularity)

    cv2.drawContours(fig_borda, [cnt], -1, (0,255, 0), 3)

    plt.figure(figsize=(15, 21))
    plt.subplot(1,2,1)
    plt.imshow(fig_th, cmap="gray")
    plt.subplot(1,2,2)
    plt.imshow(fig_borda[:,:,::-1])
    plt.show()

In [ ]:
for path in lista_ok:
    fig_borda = cv2.imread(path)
    
    # dividir a imagem em seus canais de cores
    b, g, r = cv2.split(fig_borda)

    # criar uma matriz nula para os outros canais
    zeros = np.zeros_like(b)

    # mesclar os canais para obter somente o canal azul
    blue_img = cv2.merge((b, zeros, zeros))

    x = cv2.cvtColor(blue_img, cv2.COLOR_BGR2GRAY)

    _, fig_th = cv2.threshold(x, 13, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(fig_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    cnt = max(contours, key = cv2.contourArea)
    area = cv2.contourArea(cnt)

    # Calcula o diâmetro do objeto
    _, radius = cv2.minEnclosingCircle(cnt)
    diameter = 2 * radius

    # Calcula a área de um círculo com o mesmo diâmetro do objeto
    circle_area = np.pi * (diameter/2)**2

    # Calcula a relação entre a área do objeto e a área do círculo
    circularity = area / circle_area

    print(circularity)

    cv2.drawContours(fig_borda, [cnt], -1, (0,255, 0), 3)

    plt.figure(figsize=(15, 21))
    plt.subplot(1,2,1)
    plt.imshow(fig_th, cmap="gray")
    plt.subplot(1,2,2)
    plt.imshow(fig_borda[:,:,::-1])
    plt.show()

In [ ]:
for path in lista_nok_superficie:
    fig_borda = cv2.imread(path)
    
    hsv = cv2.cvtColor(fig_borda, cv2.COLOR_BGR2HSV)
    # Define o intervalo de cores a ser removido (verde)
    lower_blue = np.array([0, 0, 50])
    upper_blue = np.array([100, 100, 255])

    # Cria uma máscara com os pixels na faixa de cores definida
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    result = fig_borda.copy()
    result[mask > 0] = [0, 0, 0] 

    x = cv2.cvtColor(result, cv2.COLOR_HSV2BGR)
    x = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    _, fig_th = cv2.threshold(x, 50, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(fig_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    cnt = max(contours, key = cv2.contourArea)
    area = cv2.contourArea(cnt)

    # Calcula o diâmetro do objeto
    _, radius = cv2.minEnclosingCircle(cnt)
    diameter = 2 * radius

    # Calcula a área de um círculo com o mesmo diâmetro do objeto
    circle_area = np.pi * (diameter/2)**2

    # Calcula a relação entre a área do objeto e a área do círculo
    circularity = area / circle_area

    print(circularity)

#     cv2.drawContours(fig_borda, [cnt], -1, (0,255, 0), 3)

#     plt.figure(figsize=(15, 21))
#     plt.subplot(1,2,1)
#     plt.imshow(fig_th, cmap="gray")
#     plt.subplot(1,2,2)
#     plt.imshow(fig_borda[:,:,::-1])
#     plt.show()

In [1]:
df.to_csv('./out.csv') 

NameError: name 'df' is not defined